<a href="https://colab.research.google.com/github/JairAlbertoHuertaDiaz45/Simulaci-n-II/blob/main/VariablesAntiteticas_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Huerta Díaz Jair Alberto**

Método de comparación Monte Carlo Acierto y Error y Crudo



In [15]:
import numpy as np
import time
import math
import random as rd
def g(x):
  return np.sqrt(np.arctan(x))

In [16]:
def estimacion(N):
  aciertos=0
  for i in range(N):  #se generan pares de números aleatorios
    x = rd.random()
    y = rd.random()
    if y <= g(x):  #ve cuales puntos quedaron debajo de la curva de la función
      aciertos += 1 #los aciertos se van acumulando
  I=aciertos/N  #Fórmula del estimador p^ = N_a / N
  return I

**Monte Carlo acierto y error**

In [17]:
def mc_acierto(N,n):  #n es el numero de estimaciones
  start_time = time.perf_counter()  #empieza el contador para el tiempo en el que se tarda
  lista=[]
  for i in range(N):
    E=estimacion(n)
    lista.append(E)
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista), np.var(lista), execution_time


estimacion(1000)

0.639

**Método de Monte Carlo Crudo**

Calculando el promedio, varianza y el tiempo promedio (datos que se ocuparán más adelante)


In [18]:
def mc_crudo(N,n):
  start_time = time.perf_counter()
  resultados_c=[] #se guardan las estimaciones
  for i in range(n): #n numero de estimaciones, comienza a hacer el bucle
    lista_c=[]
    for j in range(N):  #N es el tamaño de la muestra
      u=rd.random()
      lista_c.append(g(u))
    resultados_c.append(lista_c)
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

mean1, var1, t1 = mc_acierto(1000,100)
mean2, var2, t2 = mc_crudo(1000,100)
E = (t2 * var2) / (t1 * var1)

$$
E = \frac{t_{\text{crudo}} \cdot \mathrm{Var}(\theta_{\text{crudo}})}{t_{\text{ae}} \cdot \mathrm{Var}(\theta_{\text{ae}})}
$$

In [19]:
print("Método Acierto y Error:  media:", mean1, " varianza:", var1, " tiempo:", t1)
print("Método Crudo:            media:", mean2, " varianza:", var2, " tiempo:", t2)
print("Eficiencia (E):", E)

Método Acierto y Error:  media: 0.6324099999999999  varianza: 0.0024188919000000006  tiempo: 0.31887333499997794
Método Crudo:            media: 0.6316417945745754  varianza: 0.04313068606900262  tiempo: 0.2091265079999971
Eficiencia (E): 11.693938026089683


**PARTE II: REDUCCIÓN DE LA VARIANZA**

**Reducción de la varianza Método de Monte Carlo**

Vamos a hacer la reducción con el método de variables antipéticas, usando el teorema

$$
\text{Var}\!\left(\frac{X_1 + X_2}{2}\right)
= \frac{1}{4} \Big[ \text{Var}(X_1) + \text{Var}(X_2) + 2\,\text{Cov}(X_1, X_2) \Big]
$$

**Método de Monte Carlo Crudo**

Para este método en lugar de utilizar muestras de tamaño $u$, se usarán $1-u$

In [20]:
def mc_crudo_g_1u(N, n):
  start_time=time.perf_counter()
  resultados_c=[]
  for i in range(n): # se generan N muestras evaluando en (1-u) en lugar de u
    lista_c=[]
    for j in range(N):
      u=rd.random()
      lista_c.append(g(1-u))
    resultados_c.append(lista_c)
  end_time=time.perf_counter()
  execution_time=end_time-start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

Calculamos el promedio es decir, la reduccion de la varianza utilizando el teorema.

$
\text{Var}\!\left(\frac{f(u) + f(1-u)}{2}\right) < \text{Var}(f(u))
$

In [21]:
def mc_crudo_reduccion_var(N,n):
  start_time=time.perf_counter()
  resultados_c=[]
  for i in range(n):
    lista_c=[]
    for j in range(N):
      u=rd.random()
      lista_c.append((g(u) + g(1-u))/2.0) #promedio g(u) y g(1-u)
    resultados_c.append(lista_c)
  end_time=time.perf_counter()
  execution_time=end_time-start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

In [23]:
mean_c, var_c, t_c = mc_crudo(1000, 30)
mean_as, var_as, t_as = mc_crudo_g_1u(1000, 30)
mean_ac, var_ac, t_ac = mc_crudo_reduccion_var(1000, 30)
print("Monte Carlo crudo con u:               media =", mean_c, " varianza =", var_c, " tiempo =", t_c)
print("Monte Carlo crudo con 1-u:             media =", mean_as, " varianza =", var_as, " tiempo =", t_as)
print("Monte Carlo crudo con la red de var:   media =", mean_ac, " varianza =", var_ac, " tiempo =", t_ac)

Monte Carlo crudo con u:               media = 0.6331774412641235  varianza = 0.040779455825911144  tiempo = 0.0641894619999448
Monte Carlo crudo con 1-u:             media = 0.6304172298806665  varianza = 0.04341234931983286  tiempo = 0.061598802999924374
Monte Carlo crudo con la red de var:   media = 0.6293906001712107  varianza = 0.002768582708662282  tiempo = 0.14071783199995025


Calculamos la reducción

$
\text{Reduccion}=\!\left(\frac{Varianza Original - Varianza Nueva)}{Varianza Original}\right) * 100
$

In [24]:
reduccion=((var_c-var_ac)/var_c)*100
reduccion

np.float64(93.21083949604072)

**Reducción de la Varianza para el Método de Acierto y Error**

Utilizando variables antitéticas con pares, es decir $(u, v)$ y $(1-u, 1-v)$  

Esto funciona porque los dos indicadores están negativamente correlacionados:

$$
\text{Var}(\hat{I}) = \frac{\text{Var}(\tilde{Z})}{N/2}
= \frac{1}{N/2} \cdot \frac{1}{4} \Big[ 2\text{Var}(Z) + 2\text{Cov}(Z_1, Z_2) \Big]
$$

Si $\text{Cov}(Z_1, Z_2) < 0$, entonces:

$$
\text{Var}(\tilde{I}) < \text{Var}(\hat{I})
$$

donde

$$
\tilde{I} = \frac{1}{N/2} \sum_{k=1}^{N/2} \tilde{Z}_k,
\quad \hat{I} = \frac{1}{N} \sum_{i=1}^{N} Z_i,
\quad \tilde{Z} = \frac{Z_1 + Z_2}{2}
$$

In [25]:
def mc_acierto_reduccion_var(N, n): #n numero de estimaciones
    start_time=time.perf_counter()
    resultados=[]
    for i in range(n):  # n
        aciertos=0
        for j in range(N // 2):  # usamos N/2 porque son pares
            u1, v1 = rd.random(), rd.random()
            u2, v2 = 1 - u1, 1 - v1
            aciertos += int(v1 <= g(u1))  #incrementa aciertos
            aciertos += int(v2 <= g(u2))
        I = aciertos / N
        resultados.append(I)
    end_time = time.perf_counter()
    execution_time = (end_time - start_time) / n
    return np.mean(resultados), np.var(resultados), execution_time

In [26]:
mean, var, t = mc_acierto_reduccion_var(1000, 30)

In [27]:
print("Método Acierto y Error:                  media:", mean1, " varianza:", var1, " tiempo:", t1)
print("Método acierto y error con red de var:   media =", mean, " varianza =", var, " tiempo =", t)

Método Acierto y Error:                  media: 0.6324099999999999  varianza: 0.0024188919000000006  tiempo: 0.31887333499997794
Método acierto y error con red de var:   media = 0.6312333333333334  varianza = 8.744555555555573e-05  tiempo = 0.004857589266665249


Y utilizando la misma formula que anteriormente utilizamos para la reduccion de la varianza en el Metodo Acierto y Error.

Calculamos la reducción

$
\text{Reduccion}=\!\left(\frac{Varianza Original - Varianza Nueva)}{Varianza Original}\right) * 100
$

In [29]:
reduccion=((var1-var)/var1)*100

reduccion

np.float64(96.38489196001045)